<a href="https://colab.research.google.com/github/Dishak0411/Machine-Learning-Projects/blob/main/CNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(train_img, train_labels), (test_img, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
train_img = train_img/255.0
test_img = test_img/255.0

In [ ]:
train_img[0].shape

(28, 28)

In [ ]:
train_img = train_img.reshape(len(train_img), 28, 28, 1)
test_img = test_img.reshape(len(test_img), 28, 28, 1)

In [ ]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('cov_1_filter', min_value = 32, max_value = 128, step = 16),
          kernel_size = hp.Choice('cov_1_kernel', values = [3,5]),
          activation = 'relu',
          input_shape = (28, 28, 1)
      ),
      keras.layers.Conv2D(
          filters = hp.Int('cov_2_filter', min_value = 32, max_value = 64, step = 16),
          kernel_size = hp.Choice('cov_2_kernel', values = [3,5]),
          activation = 'relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step = 16),
          activation = 'relu'
      ),
      keras.layers.Dense(10, activation = 'softmax')
  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2, 1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters


<ipython-input-11-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [ ]:
tuner_search = RandomSearch(build_model,
                          objective = 'val_accuracy',
                          max_trials = 5,
                          directory = 'output',
                          project_name = 'Mnist Fashion')

In [ ]:
tuner_search.search(train_img, train_labels, epochs = 3, validation_split = 0.1)

Trial 5 Complete [00h 00m 27s]
val_accuracy: 0.856333315372467

Best val_accuracy So Far: 0.909500002861023
Total elapsed time: 00h 03m 17s


In [ ]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 48)        1248      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 32)        38432     
                                                                 
 flatten (Flatten)           (None, 12800)             0         
                                                                 
 dense (Dense)               (None, 112)               1433712   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 1474522 (5.62 MB)
Trainable params: 1474522 (5.62 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(train_img, train_labels, epochs=10, validation_split= 0.1, initial_epoch = 3)

Epoch 4/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1695 - accuracy: 0.9370 - val_loss: 0.2431 - val_accuracy: 0.9133
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1319 - accuracy: 0.9487 - val_loss: 0.2811 - val_accuracy: 0.9067
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1008 - accuracy: 0.9618 - val_loss: 0.2974 - val_accuracy: 0.9125
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0770 - accuracy: 0.9708 - val_loss: 0.3746 - val_accuracy: 0.9055
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0604 - accuracy: 0.9772 - val_loss: 0.3708 - val_accuracy: 0.9065
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0476 - accuracy: 0.9821 - val_loss: 0.4442 - val_accuracy: 0.9073
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0421 - accuracy: 0.9848 - val_loss: 0.4779 - val_accuracy